# MÓDULO 4: LA CIENCIA DE DATOS Y LOS MODELOS DE ANALÍTICA PREDICTIVA EN LA INDUSTRIA 4.0

## 12- Métodos probabilísticos y optimización heurística
### Bayesian inference and learning: Hidden Markov Models

### Problem Statement 
The following problem is from the Udacity course on Artificial Intelligence (Thrun and Norvig), chapter 11 (HMMs and filters). It involves a simple scenario where a person's current emotional state is determined by the weather on that particular day. The task is to find the underlying hidden sequence of states (in this case, the weather), given only a set of observations (moods) and information about state/observation changes.

In [1]:
#import required libraries
import numpy as np
import warnings
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

$P(\;Rainy\;) = P(R_{0}) = 0.5$  (initial probabilites)  
$P(\;Sunny\;) = P(S_{0}) = 0.5$  

The chances of weather changing are given as follows:  
For rainy weather, $P(S_{tomorrow}|R_{today}) = 0.4$, and $P(R_{tomorrow}|R_{today}) = 0.6$  
For sunny weather, $P(R_{tomorrow}|S_{today}) = 0.2$, therefore $P(S_{tomorrow}| S_{today}) = 0.8$  
For the purpose of formulating an HMM, we call the above ***Transition Probabilities.*** 

The corresponding mood changes, given the weather are :  
$P(H|R) = 0.4$, therefore $P(G|R) = 0.6$  
$P(H|S) = 0.9$, and $P(G|S) = 0.1$  
We call these ***Emission Probabilities***

In [2]:
S = np.array([0, 1]) # 0 Rainy, 1 Sunny
S_names = ('Rainy', 'Sunny') 
pi = np.array([0.5, 0.5]) # Initial Probabilities
O = np.array(['Happy', 'Grumpy']) # Set of observations
A = np.array([[0.6, 0.4], [0.2, 0.8]]) # {R:{R, S}, S:{R, S}} Transition Matrix
B = np.array([[0.4, 0.6], [0.9, 0.1]]) # {R: {H, G}, S: {H, G}} Emission Matrix
Y = np.array([0, 0, 1]) # 0 Happy, 1 Grumpy -- Observation sequence

### Hidden Markov Models

[HMMs](https://en.wikipedia.org/wiki/Hidden_Markov_model) are a class of probabilistic graphical models that can predict the sequence of states, given a sequence of observations that are dependent on those states, and when the states themselves are unobservable. HMMs have seen widespread success in a variety of applications, from Speech processing and Robotics to DNA Sequencing. An HMM operates according to a set of assumptions, which are :  
1. ** Markov Assumption **  
Current state is dependent on only the previous state.  
2. ** Stationarity Assumption **  
Transition probabilities are independent of time of transition.  
3. ** Independence Assumption **  
Each observation depends solely on the current underlying state (which in turn depends on the previous one), and is independent of other observations.  

An HMM is a **Generative model**, in that it attempts to find the probability of a set of observations being produced or *generated* by a class.  The parameters that we pass to the HMM class, defined below, are:  
*O* = a set of observations  
*S* = a set of states  
*A* = transition probabilities, represented as a matrix  
*B* = emission probabilities, represented as a matrix  
*pi* = initial state probabilties  
*Y* = sequence observed  

### Viterbi Algorithm

The Viterbi algorithm is a Dynamic Programming algorithm for decoding the observation sequence to uncover the most probable state sequence. Given the required parameters, it starts from the initial state and uses the transition/emission information to calculate probabilities of subsequent states. Information from the previous step is passed along to the next, similar to a belief propagation mechanism.

We store the results of each step in a table or matrix of size $k * t$, where k is the number of possible states, and t is the length of the observation sequence. The idea here is to find the path through possible states that has the maximum probability. Since initially we do not have a transition from state to state, we multiply the initial probabilities (from pi) and $P(\;observation\;|\;state\;)$ (from emission matrix B).  
Eg. For the first day, we have the observation as Happy, so:   
$P(R_{1}) = P(R_{0}) * P(H|R_{1}) = 0.5 * 0.4 = 0.2$      
$P(S_{1}) = P(S_{0}) * P(H|S_{1}) \;= 0.5 * 0.9 = 0.45$    
We log both these results in the table, since we are starting from an initial state. For the following observations, however, each state has only its maximum probability of moving to the next state logged.  

#### On Day 2 : (observation - Happy) :  
If current state = Rainy:   
$P(R_{1}) * P(R_{2}|R_{1}) = 0.20 * 0.6 = 0.12$  (given Rainy was previous state)    
$P(S_{1}) * P(R_{2}|S_{1}) = 0.45 * 0.2 = 0.09$  (Given Sunny was previous state)    
Since $0.12>0.09$, We choose $P(R_{2}|H)$ as the most probable transition from $R_{1}$, and update the table with    
$P(R_{2}|H) = P(R_{1}) * P(R_{2}|R_{1}) * P(H|R_{2}) = 0.12 * 0.4 = 0.048$    

If current state = Sunny:  
$P(R_{1}) * P(S_{2}|R_{1}) = 0.20 * 0.4 = 0.08$  (given Rainy was previous state)    
$P(S_{1}) * P(S_{2}|S_{1}) = 0.45 * 0.8 = 0.36$  (given Sunny was previous state)     
Here too, we choose $P(S_{2}|H)$ as the most probable transition from $S_{1}$, and add it to the table.      
$P(S_{2}|H) = P(S_{1}) * P(S_{2}|S_{1}) * P(H|S_{2}) = 0.36 * 0.9 = 0.324$  

  
#### On Day 3: (observation - Grumpy) :  
If current state = Rainy:   
$P(R_{2}) * P(R_{3}|R_{2}) = 0.048 * 0.6 = 0.0288$  (given Rainy was previous state)      
$P(S_{2}) * P(R_{3}|S_{2}) = 0.324 * 0.2 = 0.0648$  (given Sunny was previous state)       
As $0.0648>0.0288$, We choose $P(R_{3}|G)$ as the most probable transition from $R_{2}$, and update the table with    
$P(R_{3}|G) = P(R_{2}) * P(R_{3}|R_{2}) * P(G|R_{3}) = 0.0648 * 0.6 = 0.03888$   

If current state = Sunny:  
$P(R_{2}) * P(S_{3}|R_{2}) = 0.048 * 0.4 = 0.0192$  (given Rainy was previous state)   
$P(S_{2}) * P(S_{3}|S_{2}) = 0.324 * 0.8 = 0.2592$  (given Sunny was previous state)       
Here too, we choose $P(S_{3}|G)$ as the most probable transition from $S_{1}$, and add it to the table.    
$P(S_{3}|G) = P(S_{2}) * P(S_{3}|S_{2}) * P(G|S_{3}) = 0.2592 * 0.1 = 0.02592$  

Since now the table is completely filled, we work in reverse from probability of the last observation and its inferred state (in this case, $0.0388$ i.e Rainy) finding which state had the maximum probability up to that point. In this way, we find the most probable sequence of states corresponding to our observations!

In [14]:
class HMM:

    def __init__(self, observations, states, start_probs, trans_probs, emm_probs, obs_sequence):
        self.O = observations
        self.S = states
        self.state_names = None
        self.pi = start_probs
        self.A = trans_probs
        self.B = emm_probs
        self.Y = obs_sequence
        self.k = np.array(self.S).shape[0]
        self.t = self.Y.shape[0]
        self.table_1 = np.zeros((self.k, self.t))
        self.output_sequence = np.zeros((self.t,))
        self.fwds = None
        self.bwds = None
        self.smoothened = None

    def viterbi(self):
        # loop through states, but only for first observation
        print("Day 1 : Observation was", self.Y[0], "i.e", self.O[self.Y[0]])
        for i in range(self.k):
            self.table_1[i, 0] = self.pi[i] * self.B[i, self.Y[0]]
            print("Probability of state", i, "-->", self.table_1[i, 0])
            print("-------------------------------------------")
        print("=========================================")
        # loop through second to last observation
        for i in range(1, self.t):
            print("Day", i + 1, ": Observation was", self.Y[i], "i.e", self.O[self.Y[i]])
            # loop through states
            for j in range(self.k):
                print("If current state", j, "i.e", self.state_names[j])
                max_t1_A = 0.0
                for d in range(self.k):  # loop through states*states
                    print("probability of the previous state i.e", d, "-->", self.table_1[d, i - 1])
                    val = self.table_1[d, i - 1] * self.A[d, j]
                    print("State", d, "to State", j, "-->", self.A[d, j])
                    print(self.table_1[d, i - 1], "*", self.A[d, j], "=", val)
                    if val > max_t1_A:
                        max_t1_A = val
                    else:
                        continue
                self.table_1[j, i] = max_t1_A
                tmp = self.table_1[j, i]
                self.table_1[j, i] = self.table_1[j, i] * self.B[j, self.Y[i]]
                print("Probability of next state given previous state, transition and observation :")
                print(tmp, "*", self.B[j, self.Y[i]], "=", self.table_1[j, i])
                print("-------------------------------------------")
            print("===========================================")
        print("")
        # work backwards from the last day, comparing probabilities
        # from observations and transitions up to that day.
        for i in range(self.t - 1, -1, -1):
            max_at_i = 0.0
            max_j = 0.0
            for j in range(self.k):
                if self.table_1[j][i] > max_at_i:
                    max_at_i = self.table_1[j][i]
                    max_j = j
                else:
                    continue
                self.output_sequence[i] = j
            print("State", self.state_names[int(self.output_sequence[i])], "was most likely on day", i+1)
        print("")
        return self.output_sequence

    def get_obs(self, obs_val, emm_prob):
        ob_mat = np.zeros((self.k, self.k))
        for i in self.S:
            for j in self.S:
                if i == j:
                    ob_mat[i, j] = emm_prob[i, obs_val]
        return ob_mat

    def get_diagonal(self, mat_A, mat_B):
        x = np.transpose(mat_A).shape[1]
        mat_C = np.dot(mat_A, np.transpose(mat_B))
        mat_D = np.zeros((self.k, 1))
        for i in range(x):
            for j in range(x):
                if i == j:
                    mat_D[i][0] = mat_C[i][j]
        return mat_D

In [15]:
weather_hmm = HMM(O, S, pi, A, B, Y)
weather_hmm.state_names = S_names
obs_states = [O[i] for i in Y]
print("Observations :")
print(obs_states, "\n")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print("Using Viterbi Algorithm:\n")
    op1 = weather_hmm.viterbi()
    print("Table of state probabilities :")
    for i in weather_hmm.table_1:
        print("----------------------------")
        print("|",)
        for j in i:
            print("{0:.4f} |".format(j),)
        print("")
    print("----------------------------\n")
    op_states1 = [S_names[int(i)] for i in op1]
print(op_states1)

Observations :
['Happy', 'Happy', 'Grumpy'] 

Using Viterbi Algorithm:

Day 1 : Observation was 0 i.e Happy
Probability of state 0 --> 0.2
-------------------------------------------
Probability of state 1 --> 0.45
-------------------------------------------
Day 2 : Observation was 0 i.e Happy
If current state 0 i.e Rainy
probability of the previous state i.e 0 --> 0.2
State 0 to State 0 --> 0.6
0.2 * 0.6 = 0.12
probability of the previous state i.e 1 --> 0.45
State 1 to State 0 --> 0.2
0.45 * 0.2 = 0.09000000000000001
Probability of next state given previous state, transition and observation :
0.12 * 0.4 = 0.048
-------------------------------------------
If current state 1 i.e Sunny
probability of the previous state i.e 0 --> 0.2
State 0 to State 1 --> 0.4
0.2 * 0.4 = 0.08000000000000002
probability of the previous state i.e 1 --> 0.45
State 1 to State 1 --> 0.8
0.45 * 0.8 = 0.36000000000000004
Probability of next state given previous state, transition and observation :
0.36000000000